<a href="https://colab.research.google.com/github/van1164/Pytorch_Study/blob/main/VGGNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 10
learning_rate = 0.001

transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])


train_dataset = torchvision.datasets.CIFAR10(root='./',
                                             train=True,
                                             transform = transform,
                                             download =True)

test_dataset = torchvision.datasets.CIFAR10(root = './',
                                            train = False,
                                            transform =transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = 100,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size =100,
                                          shuffle =False)




Files already downloaded and verified


In [2]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()

        self.maxpool = nn.MaxPool2d(4)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size =3, stride =1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3,stride=1)
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(1568,10)
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x= self.relu(self.conv2(x))
        x=self.maxpool(x)
        x = x.view(x.size(0),-1)
        x=self.fc(x)

        return x
        

In [ ]:
model =VGG().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

def update_lr(optimzer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr']=lr

total_step = len(train_loader)
curr_lr = learning_rate

In [3]:
def train(epoch):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        ouputs = model(images)
        loss = criterion(ouputs,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%100 ==0:
            print("Epoch [{}/{}], step [{}/{}] Loss: {:4f}".format(epoch+1,num_epochs,i+1,total_step,loss.item()))
    
    if (epoch+1)%20 == 0:
        curr_lr /=3
        update_lr(optimizer,curr_lr)

def test():
    model.eval()
    with torch.no_grad():
        correct = 0
        total =0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data,1)
            total +=labels.size(0)
            correct +=(predicted == labels).sum().item()

        print('Accuracy of the model on the test images: {}%'.format(100*correct/total))


In [ ]:
for epoch in range(1,num_epochs):
    train(epoch)
    test()

Epoch [2/10], step [100/500] Loss: 1.871859
Epoch [2/10], step [200/500] Loss: 1.688863
Epoch [2/10], step [300/500] Loss: 1.608249
Epoch [2/10], step [400/500] Loss: 1.524811
Epoch [2/10], step [500/500] Loss: 1.521932
Accuracy of the model on the test images: 49.4
Epoch [3/10], step [100/500] Loss: 1.505223
Epoch [3/10], step [200/500] Loss: 1.393238
Epoch [3/10], step [300/500] Loss: 1.655915
Epoch [3/10], step [400/500] Loss: 1.332471
Epoch [3/10], step [500/500] Loss: 1.313959
Accuracy of the model on the test images: 55.14
Epoch [4/10], step [100/500] Loss: 1.423641
Epoch [4/10], step [200/500] Loss: 1.242416
Epoch [4/10], step [300/500] Loss: 1.394009
Epoch [4/10], step [400/500] Loss: 1.508890
Epoch [4/10], step [500/500] Loss: 1.372045
Accuracy of the model on the test images: 57.86
Epoch [5/10], step [100/500] Loss: 1.324079
Epoch [5/10], step [200/500] Loss: 1.256518
Epoch [5/10], step [300/500] Loss: 1.136456
Epoch [5/10], step [400/500] Loss: 1.349916
Epoch [5/10], step [5

2.
데이터
전처리로
정규화
(normalization)
를
수행하시오


In [4]:
import numpy as np
def calculate_norm(dataset):
    mean_ = np.array([np.mean(x.numpy(), axis=(1, 2)) for x, _ in dataset])
    mean_r = mean_[:, 0].mean()
    mean_g = mean_[:, 1].mean()
    mean_b = mean_[:, 2].mean()

    std_ = np.array([np.std(x.numpy(), axis=(1, 2)) for x, _ in dataset])
    std_r = std_[:, 0].mean()
    std_g = std_[:, 1].mean()
    std_b = std_[:, 2].mean()
    
    return (mean_r, mean_g, mean_b), (std_r, std_g, std_b)

mean, pyo = calculate_norm(train_dataset)

In [5]:
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
    transforms.Normalize(mean,pyo)
])


In [6]:
train_dataset = torchvision.datasets.CIFAR10(root='./',
                                             train=True,
                                             transform = transform,
                                             download =True)

test_dataset = torchvision.datasets.CIFAR10(root = './',
                                            train = False,
                                            transform =transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = 100,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size =100,
                                          shuffle =False)


Files already downloaded and verified


In [ ]:
model =VGG().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
for epoch in range(1,num_epochs):
    train(epoch)
    test()

Epoch [2/10], step [100/500] Loss: 1.814100
Epoch [2/10], step [200/500] Loss: 1.595370
Epoch [2/10], step [300/500] Loss: 1.643204
Epoch [2/10], step [400/500] Loss: 1.461335
Epoch [2/10], step [500/500] Loss: 1.616809
Accuracy of the model on the test images: 15.89%
Epoch [3/10], step [100/500] Loss: 1.269336
Epoch [3/10], step [200/500] Loss: 1.346250
Epoch [3/10], step [300/500] Loss: 1.289133
Epoch [3/10], step [400/500] Loss: 1.103372
Epoch [3/10], step [500/500] Loss: 1.183114
Accuracy of the model on the test images: 22.25%
Epoch [4/10], step [100/500] Loss: 1.251787
Epoch [4/10], step [200/500] Loss: 1.298890
Epoch [4/10], step [300/500] Loss: 1.202730
Epoch [4/10], step [400/500] Loss: 1.262516
Epoch [4/10], step [500/500] Loss: 1.190983
Accuracy of the model on the test images: 19.35%
Epoch [5/10], step [100/500] Loss: 0.984529
Epoch [5/10], step [200/500] Loss: 1.122375
Epoch [5/10], step [300/500] Loss: 1.203191
Epoch [5/10], step [400/500] Loss: 1.113235
Epoch [5/10], ste

3
아래
VGG
모델을
구현하고
CIFAR
-
100
데이터를
학습하시오
.
학습
결과
를
report
하시오

In [ ]:
class VGG_3(nn.Module):
    def __init__(self):
        super(VGG_3, self).__init__()

        self.block2_1= nn.Conv2d(in_channels=3, out_channels=64, kernel_size =3, stride =1)
        self.block2_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size =3, stride =1)
        self.block3_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size =3, stride =1)
        self.block3_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size =3, stride =1)
        self.block3_3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size =3, stride =1)
        
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(2048,1024)
        self.fc2 = nn.Linear(1024,10)
    def forward(self, x):
        x = self.relu(self.block2_1(x))
        x = self.relu(self.block2_2(x))
        x=self.maxpool(x)
        x = self.relu(self.block3_1(x))
        x = self.relu(self.block3_2(x))
        x = self.relu(self.block3_3(x))
        x=self.maxpool(x)
        x = x.view(x.size(0),-1)
        x=self.fc1(x)
        x=self.fc2(x)
        return x


In [ ]:
model =VGG_3().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

def update_lr(optimzer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr']=lr

total_step = len(train_loader)
curr_lr = learning_rate

In [ ]:
for epoch in range(1,num_epochs):
    train(epoch)
    test()

Epoch [2/10], step [100/500] Loss: 1.777136
Epoch [2/10], step [200/500] Loss: 1.383581
Epoch [2/10], step [300/500] Loss: 1.306512
Epoch [2/10], step [400/500] Loss: 1.478821
Epoch [2/10], step [500/500] Loss: 1.261651
Accuracy of the model on the test images: 14.68%
Epoch [3/10], step [100/500] Loss: 1.126640
Epoch [3/10], step [200/500] Loss: 1.271836
Epoch [3/10], step [300/500] Loss: 1.111669
Epoch [3/10], step [400/500] Loss: 0.956812
Epoch [3/10], step [500/500] Loss: 1.225571
Accuracy of the model on the test images: 17.1%
Epoch [4/10], step [100/500] Loss: 1.076818
Epoch [4/10], step [200/500] Loss: 1.095790
Epoch [4/10], step [300/500] Loss: 0.974246
Epoch [4/10], step [400/500] Loss: 0.634796
Epoch [4/10], step [500/500] Loss: 0.879453
Accuracy of the model on the test images: 21.73%
Epoch [5/10], step [100/500] Loss: 0.904998
Epoch [5/10], step [200/500] Loss: 0.740342
Epoch [5/10], step [300/500] Loss: 0.821802
Epoch [5/10], step [400/500] Loss: 0.929241
Epoch [5/10], step

4.
이전
문제에서
구현한
VGG
모델을
튜닝하시오

maxpool stride 2

In [ ]:
class VGG_3_2(nn.Module):
    def __init__(self):
        super(VGG_3_2, self).__init__()
        self.block2_1= nn.Conv2d(in_channels=3, out_channels=64, kernel_size =3, stride =1)
        self.block2_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size =3, stride =1)
        self.block3_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size =3, stride =1)
        self.block3_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size =3, stride =1)
        self.block3_3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size =3, stride =1)
        
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.fc1 = nn.Linear(2048,1024)
        self.fc2 = nn.Linear(1024,10)
    def forward(self, x):
        x = self.relu(self.block2_1(x))
        x = self.relu(self.block2_2(x))
        x=self.maxpool(x)
        x = self.relu(self.block3_1(x))
        x = self.relu(self.block3_2(x))
        x = self.relu(self.block3_3(x))
        x=self.maxpool(x)
        x = x.view(x.size(0),-1)
        x=self.fc1(x)
        x=self.fc2(x)
        return x

In [ ]:
learning_rate = 0.0001

model =VGG_3_2().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

def update_lr(optimzer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr']=lr

total_step = len(train_loader)
curr_lr = learning_rate

for epoch in range(1,num_epochs):
    train(epoch)
    test()

maxpool stride 3

In [9]:
class VGG_3_3(nn.Module):
    def __init__(self):
        super(VGG_3_3, self).__init__()
        self.block2_1= nn.Conv2d(in_channels=3, out_channels=64, kernel_size =3, stride =1)
        self.block2_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size =3, stride =1)
        self.block3_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size =3, stride =1)
        self.block3_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size =3, stride =1)
        self.block3_3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size =3, stride =1)
        
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2,stride=3)
        self.fc1 = nn.Linear(128,64)
        self.fc2 = nn.Linear(64,10)
    def forward(self, x):
        x = self.relu(self.block2_1(x))
        x = self.relu(self.block2_2(x))
        x=self.maxpool(x)
        x = self.relu(self.block3_1(x))
        x = self.relu(self.block3_2(x))
        x = self.relu(self.block3_3(x))
        x=self.maxpool(x)
        x = x.view(x.size(0),-1)
        x=self.fc1(x)
        x=self.fc2(x)
        return x

In [ ]:
learning_rate = 0.0001

model =VGG_3_3().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

def update_lr(optimzer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr']=lr

total_step = len(train_loader)
curr_lr = learning_rate

for epoch in range(1,num_epochs):
    train(epoch)
    test()

Epoch [2/10], step [100/500] Loss: 2.113459
Epoch [2/10], step [200/500] Loss: 2.057937
Epoch [2/10], step [300/500] Loss: 1.774823
